In [1]:
import wandb


api = wandb.Api()


In [4]:
artifact = api.artifact('hofbauer/Verbalized Multistep Confidence/multistep-HotpotQA-gpt-4.1-mini:v0')
df = artifact.get("table").get_dataframe()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb:   2 of 2 files downloaded.  


In [12]:
from utils.utils import extract_texts_and_confidences
df["steps"] = list(map(lambda x :extract_texts_and_confidences(x)[0], df["response"]))

In [18]:
from utils.datasets import get_dataset

ds = get_dataset("HotpotQA", num_samples=1000, all_collumns=True)

In [24]:
ds["context"][0]

{'title': ['Vinalhaven, Maine',
  'Owls Head, Maine',
  'North Haven, Maine',
  'Downeast Flight 46',
  'Northern California TRACON',
  'Sacramento International Airport',
  'Knox County Regional Airport',
  'Matinicus Isle, Maine',
  'Raleigh Executive Jetport',
  'Lea County Regional Airport'],
 'sentences': [['Vinalhaven is a town located on the larger of the two Fox Islands in Knox County, Maine, United States.',
   ' Vinalhaven is also used to refer to the Island itself.',
   ' The population was 1,165 at the 2010 census.',
   ' It is home to a thriving lobster fishery and hosts a summer colony.',
   ' Since there is no bridge to the island, Vinalhaven is accessible from Rockland via an approximately hour-and-fifteen-minute ferry ride across West Penobscot Bay, or by air taxi from Knox County Regional Airport.'],
  ['Owls Head is a town in Knox County, Maine, United States.',
   ' The population was 1,580 at the 2010 census.',
   ' A resort and fishing area, the community is home 

In [50]:
def supporting_facts(sample):
    topics = sample["supporting_facts"]["title"]
    titles = list(sample["context"]["title"]) 
    context_id = [titles.index(topic) for topic in topics]
    sent_id = sample["supporting_facts"]["sent_id"]
    sentences = sample["context"]["sentences"]
    supporting_sentences = [sentences[con_id][sent_id] for con_id, sent_id in zip(context_id, sent_id)]
    return supporting_sentences

    

In [51]:
supporting_facts(ds[0])

['Sacramento International Airport (IATA: SMF, ICAO: KSMF, FAA LID: SMF) is 10 mi northwest of downtown Sacramento, in Sacramento County, California.',
 'Knox County Regional Airport (IATA: RKD, ICAO: KRKD, FAA LID: RKD) is a county owned, public use airport in Knox County, Maine, United States.']

In [52]:
ds["question"][0]

'Which airport is located in Maine, Sacramento International Airport or Knox County Regional Airport?'

In [53]:
import pandas as pd
# find samples in ds by id
ds_df = ds.to_pandas()
ds_df = ds_df[["id", "supporting_facts", "context"]]
joined_df = pd.merge(ds_df, df, on="id", how="inner")

In [54]:
joined_df["supporting_sentences"] = joined_df.apply(supporting_facts, axis=1)

In [60]:
results = []

for i, row in joined_df.iterrows():
    facts = row["supporting_sentences"]
    steps = row["steps"]
    for i, step in enumerate(steps):
        for j, fact in enumerate(facts):
            # todo nli
            entailment = 1 # Placeholder for NLI check
            if entailment in [1, -1]:
                results.append({
                    "fact": fact,
                    "step": step,
                    "fact_id": j,
                    "step_id": i,
                    "entailment": entailment
                })
                break
        
    pairs = [(fact, step) for fact in facts for step in steps]
    print(pairs)
    break


[('Sacramento International Airport (IATA: SMF, ICAO: KSMF, FAA LID: SMF) is 10 mi northwest of downtown Sacramento, in Sacramento County, California.', 'Sacramento International Airport is located in Sacramento, California.'), ('Sacramento International Airport (IATA: SMF, ICAO: KSMF, FAA LID: SMF) is 10 mi northwest of downtown Sacramento, in Sacramento County, California.', 'Knox County Regional Airport is located in Knox County, Maine.'), ('Sacramento International Airport (IATA: SMF, ICAO: KSMF, FAA LID: SMF) is 10 mi northwest of downtown Sacramento, in Sacramento County, California.', 'The question asks which airport is located in Maine.'), ('Sacramento International Airport (IATA: SMF, ICAO: KSMF, FAA LID: SMF) is 10 mi northwest of downtown Sacramento, in Sacramento County, California.', 'Knox County Regional Airport'), ('Knox County Regional Airport (IATA: RKD, ICAO: KRKD, FAA LID: RKD) is a county owned, public use airport in Knox County, Maine, United States.', 'Sacramento 

In [63]:
len(pairs)
len(steps)

4

In [65]:
pairs[1]

('Sacramento International Airport (IATA: SMF, ICAO: KSMF, FAA LID: SMF) is 10 mi northwest of downtown Sacramento, in Sacramento County, California.',
 'Knox County Regional Airport is located in Knox County, Maine.')